In [1]:
import pandas as pd
import numpy as np
import requests
import os 
from dotenv import load_dotenv
import requests
from collections import Counter
from fuzzywuzzy import process, fuzz
from tqdm import tqdm
import pickle
import re
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_columns = None
pd.set_option('display.float_format', lambda x: '%.3f' % x)

import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append("../")

import src.biblioteca as bb

Note to self:
en este archivo van cosas que luego van a ser tres archivos distintos.
- Limpieza
- EDA
- Charts
- 
Recomendable separarlo y trabajarlo de forma independiente

# Input files

Base data file

In [2]:
with open('../data/pickle/dataset_usa_base.pickle', 'rb') as base_data:
    base_data = pickle.load(base_data)
print(base_data.shape)
base_data.head(1)

(455057, 9)


,title,rank,date,artist,url,region,chart,trend,streams
8246,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000


LastFM extraction

In [3]:
with open('../data/pickle/spotify_dataset_lastfm.pickle', 'rb') as data_lastfm:
    data_lastfm = pickle.load(data_lastfm)
data_lastfm.drop_duplicates(inplace=True)
print(data_lastfm.drop_duplicates().shape)
data_lastfm.head(1)

(14659, 17)


,url,artist,track,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4
8246,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert),"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap


Spotify extraction

In [4]:
with open('../data/pickle/spotify_dataset_audioFeatures.pickle', 'rb') as data_spotify:
    data_spotify = pickle.load(data_spotify)
data_spotify.drop_duplicates(inplace=True)
print(data_spotify.drop_duplicates().shape)
data_spotify.head(1)

(13989, 21)


,url,artist,track,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert),spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000


# Merging all data

In [5]:
df = base_data.merge(data_lastfm, on = 'url', indicator = True, how = 'left')
df.value_counts('_merge')

_merge
both          456306
left_only          0
right_only         0
dtype: int64

In [6]:
df.drop(['_merge'], axis = 1, inplace = True)
df = df.merge(data_spotify, on = 'url', indicator = True, how = 'left')
df.value_counts('_merge')

_merge
both          442906
left_only      13924
right_only         0
dtype: int64

In [7]:
df.drop(['artist_y', 'track_x', 'artist', 'track_y', '_merge'], axis = 1, inplace= True)
df.rename({'artist_x':'artist'}, inplace=True, axis=1)
print(df.shape)
df.head(1)

(456830, 41)


,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000,"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap,spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000


# Cleaning data

In [8]:
df.head(1)

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000,"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap,spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000


Highest rank in list + Weeks in charts

In [9]:
lista_url = df['url'].unique().tolist()
len(lista_url)

14650

In [10]:
ranks = {
    'url':[],
    'highest_rank':[],
    'weeks_in_charts':[]
}

In [11]:
for url in tqdm(lista_url):
    ranks['url'].append(url)
    ranks['highest_rank'].append(df[df['url'] == url].sort_values(by = 'rank').iloc[0,1])
    ranks['weeks_in_charts'].append(df[df['url'] == url].shape[0])

100%|██████████| 14650/14650 [13:08<00:00, 18.58it/s]


In [12]:
print(len(ranks['url']), len(ranks['highest_rank']), len(ranks['weeks_in_charts']))

14650 14650 14650


In [13]:
df = df.merge(pd.DataFrame(ranks), on = 'url', indicator= True)
df.value_counts('_merge')

_merge
both          456830
left_only          0
right_only         0
dtype: int64

In [14]:
df.drop(['_merge'], axis = 1, inplace=True)

Main music genre

Primo los géneros de las canciones -- si no hubiera datos, cojo los valores del artista

In [15]:
df.head(1)

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,highest_rank,weeks_in_charts
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000,"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap,spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000,1,429


In [16]:
df['music_genre'] = np.where(df['aritist_genre_0'].isnull() == True, df['track_genre_0'].str.lower(), df['aritist_genre_0'].str.lower())
df['music_genre'].value_counts()

hip-hop         131641
pop              65166
rap              30438
country          25113
rnb              22598
                 ...  
wave                 1
calypso              1
southern rap         1
salsa                1
c                    1
Name: music_genre, Length: 363, dtype: int64

In [17]:
cuenta_generos = Counter(genres for genres in df['music_genre'])
print(f"Hay {len(cuenta_generos)} generos diferentes.")

Hay 2976 generos diferentes.


In [18]:
dict_genres = dict(cuenta_generos.most_common(24))

In [19]:
fuzz.ratio('hip-hop', "hip hop")

86

In [20]:
def music_genres(column, genres):
    max = 0
    for key in genres.keys():
        try:
            similar = fuzz.ratio(column, key)
            if similar > max:
                max = similar
                genre = key     
        except:
            return "other"
    if max > 60:
        return genre
    else:
        return "other"

In [21]:
df["clean_music_genre"] = df.apply(lambda x: music_genres(x["music_genre"], dict_genres), axis = 1)
df["clean_music_genre"].value_counts()

hip-hop             132882
other                91631
pop                  65166
rap                  30446
country              25295
rnb                  22649
trap                 19979
electronic           11784
indie                11121
rock                  6456
soul                  5459
acoustic              3902
indie pop             3526
classic rock          3410
jazz                  3110
k-pop                 2783
house                 2610
lo-fi                 2416
indie rock            2098
latin                 2065
female vocalists      1823
black metal           1772
cloud rap             1600
reggaeton             1443
seen live             1404
Name: clean_music_genre, dtype: int64

Gender from artists

ojo, hay un poco de error -- volver a pasarlo, pero solo con el nombre de los artistas (que no sean grupos)

In [22]:
df.sample(5)

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,highest_rank,weeks_in_charts,music_genre,clean_music_genre
295675,Let It Snow! Let It Snow! Let It Snow! (with T...,190,2020-11-14,Frank Sinatra,https://open.spotify.com/track/4kKdvXD0ez7jp12...,United States,top200,NEW_ENTRY,226424.000,NaN,"27 Feb 2006, 00:10","\nFrancis Albert Sinatra (December 12, 1915 – ...","\nFrancis Albert Sinatra (December 12, 1915 – ...",jazz,swing,oldies,vocal jazz,easy listening,NaN,NaN,NaN,NaN,NaN,spotify:track:4kKdvXD0ez7jp1296JmAts,0.652,0.259,2.000,-13.336,1.000,0.059,0.903,0.000,0.362,0.836,142.169,audio_features,4kKdvXD0ez7jp1296JmAts,https://api.spotify.com/v1/tracks/4kKdvXD0ez7j...,https://api.spotify.com/v1/audio-analysis/4kKd...,155453.000,4.000,8,113,jazz,jazz
334913,Original,36,2018-07-06,"Arcangel, Bad Bunny",https://open.spotify.com/track/7MjaU5iFujwT1gO...,United States,viral50,NEW_ENTRY,NaN,NaN,"01 Jan 1970, 00:00","<a href=""https://www.last.fm/music/Arcangel,+...",,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,spotify:track:7MjaU5iFujwT1gOyvJnqNp,0.798,0.487,1.000,-4.939,1.000,0.206,0.068,0.000,0.092,0.292,151.020,audio_features,7MjaU5iFujwT1gOyvJnqNp,https://api.spotify.com/v1/tracks/7MjaU5iFujwT...,https://api.spotify.com/v1/audio-analysis/7Mja...,246360.000,4.000,13,7,NaN,other
349303,favorite crime,22,2021-07-28,Olivia Rodrigo,https://open.spotify.com/track/5JCoSi02qi3jJeH...,United States,top200,SAME_POSITION,559434.000,"27 May 2021, 16:39","14 Sep 2020, 15:39","Olivia Isabel Rodrigo is an American singer, s...","Olivia Isabel Rodrigo is an American singer, s...",pop,pop rock,female vocalists,american,indie pop,pop,indie,rock,folk,indie pop,spotify:track:5JCoSi02qi3jJeHdZXMmR8,0.369,0.272,9.000,-10.497,1.000,0.036,0.866,0.000,0.147,0.218,172.929,audio_features,5JCoSi02qi3jJeHdZXMmR8,https://api.spotify.com/v1/tracks/5JCoSi02qi3j...,https://api.spotify.com/v1/audio-analysis/5JCo...,152667.000,4.000,4,227,pop,pop
398520,Somebody’s Problem,37,2021-02-13,Morgan Wallen,https://open.spotify.com/track/42lqBjPYlmL1EXp...,United States,top200,MOVE_DOWN,420645.000,"09 Dec 2021, 16:22","14 Jan 2018, 02:26",Morgan Wallen is a country music singer and so...,Morgan Wallen is a country music singer and so...,country,racist,male vocalist,the voice,racism,NaN,NaN,NaN,NaN,NaN,spotify:track:42lqBjPYlmL1EXpMDXvhSN,0.658,0.538,5.000,-7.676,1.000,0.029,0.657,0.000,0.123,0.620,136.967,audio_features,42lqBjPYlmL1EXpMDXvhSN,https://api.spotify.com/v1/tracks/42lqBjPYlmL1...,https://api.spotify.com/v1/audio-analysis/42lq...,161773.000,4.000,3,145,country,country
31332,I Fall Apart,54,2018-09-23,Post Malone,https://open.spotify.com/track/75ZvA4QfFiZvzhj...,United States,top200,MOVE_UP,385043.000,"12 Dec 2020, 14:50","07 Jun 2015, 09:50","Austin Richard Post (born July 4, 1995), known...","Austin Richard Post (born July 4, 1995), known...",Hip-Hop,rap,trap,cloud rap,pop,Hip-Hop,rap,hip hop,soul,sad,spotify:track:75ZvA4QfFiZvzhj2xkaWAh,0.556,0.538,8.000,-5.408,0.000,0.038,0.069,0.000,0.196,0.291,143.950,audio_features,75ZvA4QfFiZvzhj2xkaWAh,https://api.spotify.com/v1/tracks/75ZvA4QfFiZv...,https://api.spotify.com/v1/audio-analysis/75Zv...,223347.000,4.000,2,1297,hip-hop,hip-hop


In [23]:
## funciona pero tiene un poco de error
def generos(col):
    for key, value in bb.dict_gender.items():
        for k in key:
            for item in value:
                try:
                    if item in col.lower():
                        return key
                except:
                    return np.nan
    return np.nan

In [24]:
def generos_2(col):
    for key, value in bb.dict_gender.items():
        try:
            if value in col.lower():
                return key
        except:
            return np.nan
    return np.nan

In [25]:
df.head(1)

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,highest_rank,weeks_in_charts,music_genre,clean_music_genre
0,Bad and Boujee (feat. Lil Uzi Vert),1,2017-01-01,Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,United States,top200,SAME_POSITION,1371493.000,"30 Jan 2018, 04:41","26 Jun 2013, 14:20","Migos is a rap trio-group from Lawrenceville, ...","Migos is a rap trio-group from Lawrenceville, ...",Hip-Hop,trap,rap,hip hop,atlanta,trap,Hip-Hop,rap,2010s,meme rap,spotify:track:0M9ydKzuF3oZTfYYPfaGX1,0.926,0.666,11.000,-5.314,1.000,0.244,0.061,0.000,0.123,0.168,127.079,audio_features,0M9ydKzuF3oZTfYYPfaGX1,https://api.spotify.com/v1/tracks/0M9ydKzuF3oZ...,https://api.spotify.com/v1/audio-analysis/0M9y...,343150.000,4.000,1,429,hip-hop,hip-hop


In [26]:
#df["gender"] = df.apply(lambda x: generos(x['content'], bb.dict_gender), axis=1)
df["gender"] = df['summary'].apply(generos)

In [27]:
df['gender'].value_counts() / df.shape[0]

male         0.497
female       0.138
group        0.073
non-binary   0.030
Name: gender, dtype: float64

In [33]:
df.sample(5)

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,highest_rank,weeks_in_charts,music_genre,clean_music_genre,gender,birthday
39899,No Limit,115,2018-05-20,"G-Eazy, A$AP Rocky, Cardi B",https://open.spotify.com/track/2DQ1ITjI0YoLFzu...,United States,top200,MOVE_UP,251989.000,NaN,"01 Jan 1970, 00:00","<a href=""https://www.last.fm/music/G-Eazy,+A$...",,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,spotify:track:2DQ1ITjI0YoLFzuADN1ZBW,0.838,0.771,1.000,-3.791,1.000,0.244,0.012,0.000,0.085,0.405,175.957,audio_features,2DQ1ITjI0YoLFzuADN1ZBW,https://api.spotify.com/v1/tracks/2DQ1ITjI0YoL...,https://api.spotify.com/v1/audio-analysis/2DQ1...,245387.000,4.000,2,277,NaN,other,NaN,NaN
279402,Sleigh Ride,89,2018-12-20,The Ronettes,https://open.spotify.com/track/5ASM6Qjiav2xPe7...,United States,top200,MOVE_UP,389683.000,"20 Dec 2022, 13:01","22 Jul 2006, 19:32",The Ronettes were a girl group of the 1960s fr...,The Ronettes were a girl group of the 1960s fr...,60s,pop,soul,oldies,motown,christmas,xmas,60s,oldies,female vocalists,spotify:track:5ASM6Qjiav2xPe7gRkQMsQ,0.529,0.772,2.000,-7.013,1.000,0.029,0.403,0.000,0.316,0.853,91.751,audio_features,5ASM6Qjiav2xPe7gRkQMsQ,https://api.spotify.com/v1/tracks/5ASM6Qjiav2x...,https://api.spotify.com/v1/audio-analysis/5ASM...,181267.000,4.000,27,9,60s,other,group,"February 11, 2009"
85328,Sorry Not Sorry,12,2017-09-28,Demi Lovato,https://open.spotify.com/track/25C5CowdsfXld2j...,United States,top200,SAME_POSITION,853844.000,"11 Jul 2017, 03:31","25 Dec 2007, 01:27","Demetria Devonne Lovato, professionally known ...","Demetria Devonne Lovato, professionally known ...",pop,pop rock,female vocalists,Disney,guilty pleasure,pop,rnb,dance,best of 2017,hino,spotify:track:0yvPEnB032fojLfVluFjUv,0.706,0.643,11.000,-6.929,0.000,0.234,0.024,0.000,0.262,0.885,143.931,audio_features,0yvPEnB032fojLfVluFjUv,https://api.spotify.com/v1/tracks/0yvPEnB032fo...,https://api.spotify.com/v1/audio-analysis/0yvP...,203760.000,4.000,1,109,pop,pop,NaN,NaN
23253,Congratulations,120,2019-03-06,"Post Malone, Quavo",https://open.spotify.com/track/3a1lNhkSLSkpJE4...,United States,top200,MOVE_UP,294999.000,NaN,"01 Jan 1970, 00:00","<a href=""https://www.last.fm/music/Post+Malon...",,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,spotify:track:3a1lNhkSLSkpJE4MSHpDu9,0.630,0.804,6.000,-4.183,1.000,0.036,0.215,0.000,0.253,0.492,123.146,audio_features,3a1lNhkSLSkpJE4MSHpDu9,https://api.spotify.com/v1/tracks/3a1lNhkSLSkp...,https://api.spotify.com/v1/audio-analysis/3a1l...,220293.000,4.000,5,1177,NaN,other,NaN,NaN
76076,Hurricane,188,2019-07-03,Luke Combs,https://open.spotify.com/track/6xHI9KjUjYT0FPt...,United States,top200,NEW_ENTRY,221769.000,"01 Aug 2018, 14:21","07 May 2014, 20:28",Luke Combs is a country music singer and songw...,Luke Combs is a country music singer and songw...,country,North Carolina,seen live,americana,american,2010s,2016,NaN,NaN,NaN,spotify:track:6xHI9KjUjYT0FPtGO8Mxa1,0.464,0.813,10.000,-6.185,1.000,0.042,0.015,0.000,0.254,0.515,75.977,audio_features,6xHI9KjUjYT0FPtGO8Mxa1,https://api.spotify.com/v1/tracks/6xHI9KjUjYT0...,https://api.spotify.com/v1/audio-analysis/6xHI...,222533.000,4.000,77,383,country,country,male,"May 15, 2017"


In [35]:
df[df['artist'] == 'Demi Lovato'].to_csv('test.csv', sep = ';')

Age from artist

-- taking the data from the summary

In [29]:
df['birthday'] =  df['content'].apply(lambda x: re.findall(r'\w* \d*, \d{4}',str(x))).str[0]
df['birthday'].value_counts()

December 2, 1998     10839
July 4, 1995         10700
January 23, 1998      9272
October 24, 1986      8719
December 18, 2001     7645
                     ...  
April 14, 1993           1
May 6, 2008              1
 1970, 1971              1
August 19, 1986          1
August 30, 1997          1
Name: birthday, Length: 999, dtype: int64

In [30]:
df.sample()

,title,rank,date,artist,url,region,chart,trend,streams,published_track,published,summary,content,aritist_genre_0,aritist_genre_1,aritist_genre_2,aritist_genre_3,aritist_genre_4,track_genre_0,track_genre_1,track_genre_2,track_genre_3,track_genre_4,URI,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature,highest_rank,weeks_in_charts,music_genre,clean_music_genre,gender,birthday
33662,Ric Flair Drip (& Metro Boomin),197,2019-08-26,Offset,https://open.spotify.com/track/7sO5G9EABYOXQKN...,United States,top200,MOVE_DOWN,206063.000,"06 May 2021, 23:54","10 Nov 2007, 03:21","Kiari Kendrell Cephus (born December 14, 1991)...","Kiari Kendrell Cephus (born December 14, 1991)...",trap,Hip-Hop,rap,ambient,techno,rap,trap,Hip-Hop,hip hop,Awesome,spotify:track:7sO5G9EABYOXQKNPNiE9NR,0.880,0.428,9.000,-8.280,1.000,0.206,0.149,0.000,0.114,0.333,100.007,audio_features,7sO5G9EABYOXQKNPNiE9NR,https://api.spotify.com/v1/tracks/7sO5G9EABYOX...,https://api.spotify.com/v1/audio-analysis/7sO5...,172800.000,4.000,2,1974,trap,trap,group,"December 14, 1991"


# EDA

# Questions / hypothesis

Most streamed artists

In [ ]:
top10 = df.groupby(['artist'])['streams'].sum().reset_index().sort_values(by = 'streams', ascending = False).head(10)
top10['ratio'] = top10['streams'] * 100 / (df[~df['streams'].isnull()]['streams'].sum())
top10

Most streamed music genres

In [ ]:
top10genre = df.groupby(['aritist_genre_0'])['streams'].sum().reset_index().sort_values(by = 'streams', ascending = False).head(10)
top10genre['ratio'] = top10genre['streams'] * 100 / (df[~df['streams'].isnull()]['streams'].sum())
top10genre[top10genre['ratio'] > 0]

Artist evolution

In [ ]:
lista_artistas = list(top10['artist'])
df_top10 = df[df['artist'].isin(lista_artistas)]
df_top10.head()

In [ ]:
num_rows = 20
years = list(range(1990, 1990 + num_rows))
data_preproc = pd.DataFrame({
    'Year': years, 
    'A': np.random.randn(num_rows).cumsum(),
    'B': np.random.randn(num_rows).cumsum(),
    'C': np.random.randn(num_rows).cumsum(),
    'D': np.random.randn(num_rows).cumsum()})

In [ ]:
sns.lineplot(x='Year', y='value', hue='variable', 
             data=pd.melt(data_preproc, ['Year']))

In [ ]:
test = pd.pivot_table(df_top10[df_top10['chart'] == 'top200'][['date', 'artist', 'rank']], index = 'date', values = ['rank'], columns = 'artist').reset_index().droplevel(level=0, axis =1)
test.rename({'':'date'}, axis = 1, inplace=True)
print(test.shape)
test.head()

In [ ]:
sns.lineplot(x='Year', y='value', hue='variable', 
             data=pd.melt(data_preproc, ['Year']))

In [ ]:
sns.set(rc={'figure.figsize':(50,8.27)})

c = sns.lineplot(data = test, x = test['date'].unique().tolist(), y = 'Ariana Grande')
c.set_xticklabels(labels = test['date'].unique().tolist(), rotation=90)

Music genre evolution

In [ ]:
df

Genre distibution

How was the music during covid